In [111]:
import pandas as pd
import warnings
import tkinter as tk

warnings.filterwarnings('ignore')

In [182]:
## Importing the Data from Files.

cols = 'user_id item_id rating timestamp'.split()
df = pd.read_csv('u.data',sep='\t',names=cols)
cols_movies = ['item_id', 'title']+[str(i) for i in range(22)]
movie_titles = pd.read_csv('u.item',sep='|',names=cols_movies, encoding='latin-1')
movie_titles= movie_titles[['item_id','title']]

## Creating a Pivot table after merging both the data.

data = pd.merge(df,movie_titles, on='item_id')
rating = pd.DataFrame(data.groupby('title')['rating'].mean())
rating['count'] = data['title'].value_counts()
pivot_df = data.pivot_table(index = 'user_id', columns = 'title', values = 'rating')

## Defining Function 

def recommendate():
    movie = movie_var.get().lower().strip()
    if movie:
        try:
            movie = movie_titles['title'][movie_titles['title'].apply(lambda x: movie in x.lower())].values[0]
            print(movie)
            movie_var.set(movie)
        except IndexError:
            recommend.set("Movie Not Found")
            also_try.set("Movie Not Found")
        else:

            corr_df = pd.DataFrame(pivot_df.corrwith(pivot_df[movie]), columns = ['Correlation'])
            corr_df.dropna(inplace=True)
            corr_df = corr_df.join(rating['count'])
            recommend.set(corr_df[(corr_df['count']>200)&(corr_df['Correlation']>0.4)].sort_values(by='Correlation',ascending=False).index[1])
            also_try.set(corr_df[corr_df['Correlation']>0.4].sort_values(by="Correlation",ascending=False).index[0]) 


## DESIGNING GUI FOR RECOMENDATION SYSTEM

app = tk.Tk()
app.title('Recommendation System')
app.geometry('450x350')
app.configure(bg='#567')

## Creatings Labels

tk.Label(app,text='Recommendation System',font=('georgia', 15 ,"bold"),bg='black',fg='red').place(x=20,y=10)
tk.Label(app, text='Movie you watched',bg='White',fg='black',font='Georgia').place(x=50,y=50)
tk.Label(app, text='Recommended for You',font='Georgia',fg='black').place(x=50,y=125)
tk.Label(app, text='Also Try',font='Georgia',fg='black').place(x=50,y=150)

## Declaring Variables for Labels

movie_var=tk.Variable(app)
tk.Entry(app,textvariable = movie_var,width=30,fg='black').place(x=250,y=50)

recommend = tk.Variable(app)
recommend.set("Type a movie name...")
tk.Label(app, textvariable=recommend,width=25,fg='black',bg='white').place(x=250,y=125)
also_try = tk.Variable(app)
also_try.set("Type a movie name...")
tk.Label(app, textvariable=also_try,width=25,fg='black',bg='white').place(x=250,y=150)

## Creating Button

tk.Button(app,text='Find Suggestion',font='Algerian',bg='Black',width=15,fg='White',command=recommendate).place(x=150,y=82)

## Infinite loop

app.mainloop()